
---
<big><big><big><big><big><big>Sieci neuronowe 2018</big></big></big></big></big></big>

---





---
<big><big><big><big><big>Sieci rekurencyjne</big></big></big></big></big>

---



---

<id=tocheading><big><big><big><big>Spis treści</big></big></big></big>
<div id="toc"></div>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="nn_figures/" width="100%">

# Sieci rekurencyjne
<img src="nn_figures/rnn.jpeg" width="90%"> [Nature]

1. __rozwinięcie sieci__ w $n$ składowych dla wygenerowania $n$-elementowego ciągu
2. $x_t$ to wejscie w chwili $t$
  * gdy generujemy więcej niż jedno słowo (znak) wprzód, to poprzednio wygenerowane staje się wejściem do następnego
3. __pamięć__ $s_t$ 
  * obliczane na podstawie poprzednich $s_t=f(Ux_t+Ws_{t-1})$
  * $f$ funkcją nieliniową
4. __wyjście__ $o_t$ w chwili $t$
  * zwykle jako $softmax(Vs_t)$
  * zwraca wektor prawdopodobieństw stanów dyskretnych
  * interesujący może być np. tylko ostatni stan określający znaczenie zdania (sentiment analysis)
5. $s_t$ przechowuje __całą__ informację na temat poprzednich stanów obliczeń
  * praktycznie nie jest wystarczająca
6. __wszystkie__ kroki __dzielą__ te same parametry $U, V, W$


## Problemy RNN
1. od dawna znane różne podstawowe architektury RNN
  * stanem pamięci jest stan ukryty i tam następuje rekurencja
  * aktualny stan wyjsciowy staje się _dodatkowym_ stanem wejściowym (jak w automatach)
2. podstawowymi problemami są
  * pamięć jedynie ostatnich akcji, _zapominanie_ stanów poprzednich
  * pamięć jedynie pojedynczych stanów globalnych dla całego modelu bez pamięci stanów ostatnich
  * stąd potrzeba modelu wypełniającego tą dziurę - __long-short time memory__
  * eksplodujące / zanikające gradienty

## Zastosowania
1. __modelowanie i generowanie języka__
  * predykcja __prawdopodobieństwa__, że zdanie jest poprawne
  * samplując z tego dostajemy model __generatywny__
  * model językowy z użyciem __n-gramów__
  $$P(w_1,\dots,w_m)=\prod_{i=1}^mP(w_i\mid w_1,\dots,w_{i-1})\approx\prod_{i=1}^mP(w_i\mid w_{i-(n-1)},\dots,w_{i-1})$$
  dla n-gramów $$P(w_i\mid w_{i-(n-1)},\dots,w_{i-1})=\frac{\#(w_{i-(n-1)},\dots,w_{i-1}, w_i)}{\#(w_{i-(n-1)},\dots,w_{i-1})}$$
2. __tłumaczenie języka__
  * podobne do modelowania
  * wymaga zwykle przeczytania kompletnego zdania w jednym języku __przed__ wygenerowaniem pierwszego słowa nowego zdania
3. __rozpoznawanie języka__
  * wejściem są odczytane __fonemy__
  * wyjściem nowe fonemy lub transkrypcja na zdania (tłumaczenie)
4. Modele RNN pozwalają przyjmować wejścia o __zmiennej długości__
  * na przykład opis obrazu jako wiele losowych sampli z niego

<img src="nn_figures/rnn-diagrams.jpeg" width="80%"> [Karpathy]
* od lewej do prawej
  * zwykłe przetwarzanie 
  * tłumaczenie pojedynczego obiektu na opis (na przykład obraz na wiele związanych znim słów)
  * __analiza sentymentu__ przyjmuje całą sekwencję i ocenia ją na końcu
  * tłumczaenie maszynowe
  * równoległe wejście-wyjście

## Back-Propagation Through Time BPTT
1. za każdym razem patrzymy kilka kroków wstecz

In [4]:
# klasa RNN (za http://wildml.com)
class RNNNumpy():
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), 
                                   (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), 
                                   (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), 
                                   (hidden_dim, hidden_dim))
        
    def forward_propagation(self, x):
        # The total number of time steps
        T = len(x)
        # During forward propagation we save all hidden states in s because need them later.
        # We add one additional element for the initial hidden, which we set to 0
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step. Again, we save them for later.
        o = np.zeros((T, self.word_dim))
        # For each time step...
        for t in np.arange(T):
            # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
            s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
 
    def predict(self, x):
        # Perform forward propagation and return index of the highest score
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
 
    #RNNNumpy.predict = predict
    #RNNNumpy.forward_propagation = forward_propagation


## BPTT
<img src="nn_figures/rnn.jpeg" width="70%"> [Nature]
1. w każdym kroku należy znaleźć wszystkie macierze parametrów $U, V, W$
  * są wspólne dla wszystkich kroków
  * zwykle mają dużo parametrów
  * niech będzie $N$ różnych słów, a pamięć jest reprezentowana przez wektor o długosci $K$
    * $x_t\in\mathbb{R}^{N}$
    * $o_t\in\mathbb{R}^{N}$
    * $s_t\in\mathbb{R}^{K}$
    * $U\in\mathbb{R}^{K\times{}N}$
    * $V\in\mathbb{R}^{N\times{}K}$
    * $W\in\mathbb{R}^{K\times{}K}$
2. parametry są __dzielone__ we wszystkich przewidywanych krokach
  * gradient w aktualnym kroku zależy 
    * od obliczeń w aktualnym kroku czasu
    * od obliczeń w poprzednim kroku
  * odpowiada to wykorzystaniu __reguły łańcuchowej__

In [5]:
# za [Britz]
def bptt(self, x, y):
    T = len(y)
    # wykonanie propagacji wprzód (zwraca ostatnie wyjscie i stan pamięci)
    #  forward_propagation() wykonuje kroki wprzód zapamiętując wszystkie wartosci pośrednie,
    #  które będą później potrzebne
    o, s = self.forward_propagation(x)
    # macierze potrzebne dla akumulacji gradientów
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # teraz cofając się wstecz w obliczeniach
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # wstęczne obliczenia dla ostatniego kroku
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # wsteczna propagacja w czasie po poprzedzających krokach, ale co najwyżej bptt_truncate kroków
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # aktualizacja
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

## BPTT
1. Algorytm jest w stanie nauczyć się prostych zależności
  * kolejność słów: bi-gramy, tri-gramy
  * częstość występowania słów
  * prostej składni
  * prostej interpunkcji
3. Jednak
  *
  * podawane zdania są zbyt krótkie by nauczyć poprawnej gramatyki
  * dłuższe zdania znacznie zwiększają złożoność uczenia
  * __nie jest w stanie__ nauczyć się zależności między __odległymi__ słowami
    * proste RNN są w stanie imitować __jedynie__ pamięć krótko-terminową
  * BPTT cierpi w dużym stopniu na problem zanikającego / eksplodującego gradientu

### BPTT koszt i wsteczna propagacja
<img src="nn_figures/rnn.jpeg" width="70%"> [Nature]

<img src="nn_figures/rnn-bptt1.png" width="70%"> [Nature]

1. koszt
$$E(y, \widehat{y})=\sum_tE_t(y_t,\widehat{y}_t)$$
2. dla $z_3=Vs_3$ mamy
$$\begin{align}
\frac{\partial E_3}{\partial V} &= \frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial\widehat{y}_3}{\partial V}\\
&=\frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial\widehat{y}_3}{\partial z_3} \frac{\partial z_3}{\partial V}\\
&=(\widehat{y}_3-y_3)\otimes s_3
\end{align}$$
3. dla pochodnej po $W$ zaczyna się pojawiać rekurencja
$$\begin{align}
\frac{\partial E_3}{\partial W} &= \frac{\partial E_3}{\partial s_3} \frac{\partial s_3}{\partial W}\\
&= \frac{\partial E_3}{\partial \widehat{y}_3}\frac{\partial \widehat{y}_3}{\partial s_3} \frac{\partial s_3}{\partial W}\\
&\hskip3em\text{jednak $s_3$ bezpośrednio zależy od $s_2$, które nie jest stałe!}\\
s_3&=\tanh(U x_t+W s_2)\\
\frac{\partial E_3}{\partial W} &=\sum_{t=0}^3\frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial \widehat{y}_3}{\partial s_3} \frac{\partial s_3}{\partial s_t}\frac{\partial s_t}{\partial W}\\
\end{align}$$
<img src="nn_figures/rnn-bptt-gradients.png" width="70%"> [Nature]
4. w rzeczywistości BPTT niewiele się różni od zwykłej wstecznej propagacji
  * w sieci warstwowej parametry między warstwami __nie są__ dzielone
  * nie ma potrzeby ich sumowania
  * w analogiczny sposób można zdefiniować regułę delta
  $$\delta^3_2=\frac{\partial E_3}{\partial z_2}=\frac{\partial E_3}{\partial s_3}\frac{\partial s_3}{\partial s_2}\frac{\partial s_2}{\partial s_2}$$

### BPTT i zanikający gradient
1. podstawowym problemem w uczeniu jest zanikanie gradientu
  * problem zauważył Hochreiter, który był autorem modelu LSTM
$$\frac{\partial E_3}{\partial W} =\sum_{t=0}^3\frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial \widehat{y}_3}{\partial s_3} \frac{\partial s_3}{\partial s_t}\frac{\partial s_t}{\partial W}$$
2. w rozwiązaniu występuje czynnik
$$\frac{\partial s_3}{\partial s_t}$$
  * i tak chociażby $$\frac{\partial s_3}{\partial s_1} = \frac{\partial s_3}{\partial s_2}\frac{\partial s_2}{\partial s_1}$$
  * skąd mamy
  $$\frac{\partial E_3}{\partial W} =\sum_{t=0}^3\frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial \widehat{y}_3}{\partial s_3} \left(\prod_{j=t+1}\frac{\partial s_j}{\partial s_{j-1}}\right)\frac{\partial s_t}{\partial W}$$
  * $s_t=\tanh(Ux_t+Ws{t-1})$
  * $\tanh()$ ma obszar saturacji po lewej i prawej stronie, a jego gradient maleje __eksponencjalnie__ szybko
  * jeśli aktywacje są daleko od własciwych, to gradient spada prawie do zera
  * wymnażanie bardzo małych wartosci tylko eksponencjalnie szybko je jeszcze zmniejsza...
3. eksplodujący gradient pojawia się równie często
  * jest efektem kilku wysokich aktywacji
  * może prowadzić do oscylacji, gdy nadchodzące sygnały są sprzeczne
  * w miarę łatwo sobie z nim poradzić przez obcinanie gradientu z wysoką normą
4. a jak z zanikającym gradientem?
  * trudniej: sieć nie uczy się wale albo potrzebuje wykładniczo wiele czasu
  * poprawna inicjalizacja
  * ReLU zamiast funkcji sigmoidalnych

# Problemy
1. __krótka__ a __długa__ pamięć
  * RNN z algorytmem typu BPTT szybko ___zapomina___ informacje
  * korzysta tylko z ostatniej
  * model dla angielskiego na poziomie znaków szybko nauczy się, że po znaku `q` __zawsze__ występuje znak `u`
  * jednak nie nauczy się informacji kontekstowej z poprzedniego zdania

# Long Short Time Memory LSTM
1. w 1991 Sepp Hochreiter obronił pracę dyplomową w Monachium w której 
  * przedstawił szczegółową analizę uczenia sieci rekurencyjnych
  * odkrył zjawisko zanikajacego i eksplodujacego gradientu
2. w 1997 zaproponował, wraz z Jurgenem Schmidhuberem, model LSTM 
  * Neural Computation:9(8):1735-1780 
  * wcześniej w 1995 w _technical document_ w Monachium
3. model LSTM stał się początkiem dla wielu innych modeli
  * rozwinięcia LSTM, np. GRU (dodatkowe bramki, inny przepływ, etc.)
  * sieci warstwowe typu Highway
  * sieci ResNet

## LSTM (za Hochreiterem)
1. jako podstawowy problem Hochreiter zauważył ___zanikający sygnał błędu___
  * błąd zanika i sieć nie uczy się niczego w sensownym czasie
  * dla sigmoidalnych funkcji aktywacji wagi musiałyby być większe od $\approx4$ by sygnał miał wystarczającą wartość
  * jednak większych przy inicjalizacji wagi nic nie pomogą, bo odpowiednia pochodna maleje jeszcze szybciej
  * BPTT jest bardzo czuły na ostatnie zmiany/sygnały
  * Hochreiter zauważył, że konieczne jest zapewnienienie stałego przepływu sygnału błędu
    * wniosek: funkcja aktywacji __musi być liniowa__
2. komórka LSTM

<img src="nn_figures/LSTM-Hochreiter.pdf" width="80%"> [Neural Computation]

<img src="nn_figures/gers_lstm.png" width="80%"> [Hochreiter]

  * dodaje (multiplikatywną) __bramkę wejściową__ $in$, która ma zabezpieczyć zawartość pamięci od wpływu _nieistotnych_ wejść
  * analogicznie __bramkę wyjściową__ $out$ mającą zabezpieczyć inne komórki przed wpływem (aktualnych) nieistotnych informacji w komórce
  * __czemu bramki ?__
    * bramka wejsciowa $in$ kontroluje przepływ sygnału błędu by zabezpieczyć przed konfliktami wag
      * czasem komórka _powinna_ uzyć wejścia z innej komórki
      * czasem nie
      * bramka wejsciowa kontroluje to
    * podobnie bramka $out$ kontroluje wagi wyjściowe
    * bramki wejściowa/wyjściowa muszą się __nauczyć__, które sygnały wyłapać/zablokować

### LSTM przykład (za Hochreiterem)
<img src="nn_figures/LSTM-Hochreiter-flow.pdf" width="80%"> [Neural Computation]


## LSTM krok po kroku (za Christopher Olah)
### bramka wejściowa

<img src="nn_figures/LSTM3-focus-f.png" width="80%"> [Colah]
  * sigmoidalna bramka wejsciowa decyduje __które__ informacje będą aktualizowane
    * wartość poprzedniej pamieci jest __wymnażana__ przez $f_t$
    * $f_t$ jest bramką _zapominającą_ nieistotne w tej chwili informacje

### aktualizacja stanu
<img src="nn_figures/LSTM3-focus-i.png" width="80%"> [Colah]
  * komórka decyduje które wartości z wejścia należy _dodać_ do tej aktualizowanej
    * wartości $i_t\widetilde{C}_t$ są dodawane do poprzednio wyczyszczonej i odpowiednio przeskalowane
    * to decyduje, które informacje z wejścia (i stanu pamięci) należy teraz użyć
  * te wartości połączone decydują o wyjsciu stanu komórki


<img src="nn_figures/LSTM3-focus-C.png" width="80%"> [Colah]

### wyjście

<img src="nn_figures/LSTM3-focus-o.png" width="80%"> [Colah]
  * po pierwsze bramka __wyjściowa__ (sigmoidalna) decydująca, które elementy stanu należy przekazać na wyjście $o_t$
  * stan komórki jest reskalowany do $(-1,+1)$ przez $\tanh$
  * przeskalowany stan komórki jest filtrowany przez bramkę wyjsciową
  
### warianty
* bramki wykorzystują wgląd w stan komórki

<img src="nn_figures/LSTM3-var-peepholes.png" width="80%"> [Colah]
  * nie zawsze wykorzystywane przez wszystkie bramki

* połączenie bramek zapominającej i wejsciowej

<img src="nn_figures/LSTM3-var-tied.png" width="80%"> [Colah]
  * zapomina jedynie te składniki, w które zostanie wstawiona nowa informacja

* Gated Recurrent Unit (Cho, 2014)

<img src="nn_figures/LSTM3-var-GRU.png" width="80%"> [Colah]
  * spore uproszczenie, a przez to staje się popularna
  * połączenie bramek zapiminajacej i wejsciowej
  * łączy stan komórki $C_t$ wraz ze stanem ukrytym komórki $h_t$

## Jakie parametry?
1. uczenie sieci LSTM nie jest trudne
  * jest bardzo wiele wariantów
  * architektura nie jest oczywista i ma wiele składników możliwe, że na wiarę
  * które są najlepsze i jak je uczyć?
2. R. Jozefowicz, Zaremba i Sutskever wykonali duży przegląd możliwych architektur wyciągając szereg wniosków
  * istnieje wiele architektur podobnych do GRU lepszych na wielu zadaniach
  * LSTM: jest generalnie najlepszy jeśli użyty był Dropout
  * LSTM: dodawanie jedynki do bramki zapominającej zwykle minimalizuje różnicę do najlepszych architektur
    * typowa inicjalizacja ustawia wszystkie wagi na małe wartości
    * inicjalizacja biasu bramki zapominajacej na małą wartość sprzyja pojawieniu się zanikającego gradientu
    * rozwiązaniem może być inicjalizacja go na wyższe wartości rzędu $1 - 2$
  * istotność bramek
    * bramka wejściowa __jest__ istotna
    * bramka wyjściowa __nie jest__ istotna
    * bramka zapominająca __jest bardzo__ istotna dla wszystkich problemów __poza__ modelowaniem języka
3. autorzy wykorzystali prostą procedurę przeszukiwania
  * utrzymywali listę 100 znalezionych najlepszych architektur poszukując dla nich najlepszych hiperparametrów
  * w każdym etapie wykonuje jeden z kroków
    * losuje jedną ze 100 architektur
      * ewaluuje 20 losowych ustawień hiperparametrów dla każdego z 3 zadań
      * ocenia przez
      $$\min\frac{\text{najlepsza dokładność dla architektury dla zadania}}{\text{najlepszy wynik GRU dla zadania}}$$
      przy czym GRU były wyliczone dla wszystkich dozwolonych architektur
    * wybiera jedną ze 100 architektur
      * mutuje parametry
  * autorzy
    * ewaluowali 10 tysięcy różnych architektur
    * 1000 z nich przeszło początkowy test zapamiętywania
      * 5 znaków w sekwencji, dla wszystkich 26 możliwości
      * jest czytanych w sekwencji
      * i ma być odtworzone w tej samej sekwencji
    * te 1000 architektur było sprawdzone dla średnio 2200 konfiguracji
    * razem wykonali testy dla ok. 230 tysiecy konfiguracji
    * zadania
      * obliczenie sumy (w postaci znaków) dla sekwencji składajacej się z dwóch sekwencji
      * predykcja następnego znaku w kodzie XML
      * modelowanie języka Penn Tree-Bank
      * (dodatkowo) modelowanie muzyki polifonicznej
      
  * rezultaty
    * GRU było lepsze od LSTM na wszystkich zadaniach poza modelowaniem języka
    * jeśli wykorzystano dropout, to LSTM było zdecydowanie najlepsze dla problemów modelowania jezyka
    * LSTM z wysokim biasem bramki zapominajęcej było lepsze od innych LSTM i GRU na prawie wszystkich zadaniach
    * trzy mutacje znalezione okazały się konkurencyjne do innych modeli

## Wielkość ukrytej warstwy
1. $x_t$ i $h_t$ są wektorami
  * jeśli rozpoznajemy znaki, to $h_t$ będzie wektorem prawdopodobieństw znaków
  * w przypadku słów, trzeba wykorzystać jakiś word-embedding, by zredukować wymiarowość
    * word-embedding pozwoli reprezentować słowa jako wektory w $\mathbb{R}^K$
    * word-embedding może być uczony __w trakcie__ uczenia modelu
2. modele LSTM zawierają pojęcie __hidden layer__
  * warstwa ukryta jest zbiorem neuronów w bramkach zapominajacych, wejsciowych, wyjsciowych, etc.
  * ich wielkość to właśnie warstwa ukryta
  * większa warstwa ukryta zapewnia modelowi większą pojemność
3. innym typowym parametrem jest żądanie by model zwracał całe sekwencje
  * w Keras to parametr `return_sequences`
 
  <img src="nn_figures/Keras-LSTM-return-sequences.png" width="80%"> [Chollet]
  * to pozwala budować sieci __wielopoziomowe__
  <img src="nn_figures/LSTM-two-layer.png" width="80%"> [Colah]


